# Welcome to Tecton!

We've designed this notebook to introduce you to the basic workflow of creating a feature view in Tecton, testing it, and pushing it to your Tecton instance when you're done.

# Prerequisites

Before proceeding, do the following in a terminal:

* Install a Python 3.8 interpreter. We need Python 3.8 since Snowflake's Snowpark module doesn't yet support 3.9+. We recommend using a fresh virtual environment for Tecton.
* Install Tecton and pip requirements: `pip install -r requirements.txt`. We also provide a command to install the packages directly from this notebook in the "Installation" section below
* Create a file called `.env` in the same folder as this Jupyter notebook consisting of the following two lines. Substitute "user" and "password" for your username and password on the Snowflake workspace to which you will be connecting. Do not include quote marks.
```ini
SNOWFLAKE_USER=user
SNOWFLAKE_PASSWORD=password

# Optionally include the next line if you want to run the real-time features REST API section
REST_API_KEY=key
```
* Log in to your Tecton instance using the CLI, for example: `tecton login https://mycluster.tecton.ai`
* Stop this Jupyter notebook session, and restart it

# Installation

You'll only need to run this once, but this will install the right packages for you to use Tecton on Snowflake.
We suggest that you use a fresh virtual environment for this. Alternatively, you can run `pip install -r requirements.txt`
to install from the requirements file.

In [ ]:
import sys
!{sys.executable} -m pip install 'tecton[snowpark]==0.6.0rc1' python-dotenv pandas snowflake-snowpark-python snowflake-connector-python

### Issues with cffi library

For certain Mac Apple Silicon users using Homebrew's libcffi, you may need to force MacOS to use a different `_cffi` library. 

1. Take note of the folder where Python is trying to load this module from, e.g. `~/.virtualenvs/tecton/lib/python3.8/site-packages/`.

2. Run this command to download a fixed version: 
```sh
$ curl https://tecton.ai.public.s3.us-west-2.amazonaws.com/training/_cffi_backend.cpython-38-darwin.so -o <folder>/_cffi_backend.cpython-38-darwin.so
```

3. Stop your Jupyter notebook environment and restart it

# Initializing your session

## Logging into Tecton

Load the Snowflake username and password environment variables

In [3]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [4]:
import tecton
import logging
import datetime
import pandas as pd

# Tell Tecton you're on Snowflake and that you want Tecton to validate your logic and data sources
tecton.conf.set("ALPHA_SNOWFLAKE_COMPUTE_ENABLED", True)
tecton.set_validation_mode("auto") # Defaults to "explicit"

# Disable logging for snowflake connector and snowpark
logging.getLogger('snowflake.connector').setLevel(logging.WARNING)
logging.getLogger('snowflake.snowpark').setLevel(logging.WARNING)

tecton.version.summary()

Version: 0.6.0rc1
Git Commit: 8ad2ad31cc64df8768ae78815281c45532b53529
Build Datetime: 2023-02-28T02:40:38


In [ ]:
# Uncomment this if you have an API key you want to use
# tecton.set_credentials(tecton_url='https://dev-snowflake.tecton.ai/api', tecton_api_key='<your api key>')

# Use this call if you want to use the credentials already established via 'tecton login'
tecton.set_credentials(tecton_url='https://dev-snowflake.tecton.ai/api')

tecton.test_credentials()
print(tecton.who_am_i())

## Logging into Snowflake

In [6]:
import os
import snowflake.connector

connection_parameters = {
    'user': os.environ['SNOWFLAKE_USER'],
    'password': os.environ['SNOWFLAKE_PASSWORD'],
    'account': 'tectonpartner',         # Replace with your own account
    'warehouse': 'DEMO_WH',             # Replace with your own warehouse
    'database': 'TECTON_DEMO_DATA',     # Replace with your own database
    'schema': 'FRAUD',                  # Replace with your own schema
}
conn = snowflake.connector.connect(**connection_parameters)
tecton.snowflake_context.set_connection(conn) # Tecton will use this Snowflake connection for all interactive queries

In [7]:
# Quick helper function to query snowflake from a notebook
# Make sure to replace with the appropriate connection details for your own account
def query_snowflake(query):
    df = conn.cursor().execute(query).fetch_pandas_all()
    return df

# Create feature view from scratch

### Create data source (canned)

In this case, we are creating a data source that consists of hardcoded values. You can load from Snowflake either via a `query` or a `table`. You just need `SELECT` access to whatever table you're loading from.

In [8]:
transactions_batch = tecton.BatchSource(
    name='transactions_batch',
    
    batch_config=tecton.SnowflakeConfig(
      query="""
        SELECT USER_ID, TIMESTAMP, AMOUNT
        FROM (
          VALUES ('user_469998441571', TO_TIMESTAMP('2022-04-09 01:00:00'), 100), 
                 ('user_469998441571', TO_TIMESTAMP('2022-04-10 01:00:00'), 200), 
                 ('user_445755474326', TO_TIMESTAMP('2022-04-09 01:00:00'), 300), 
                 ('user_445755474326', TO_TIMESTAMP('2022-04-10 01:00:00'), 400) 
          AS DATA_VALUES (NAMEORIG, TIMESTAMP, AMT)
        )""",
        timestamp_field="TIMESTAMP"
    )
)

### Create data source (Snowflake)

Loading a table from Snowflake (though this could also be a query)

In [8]:
transactions_batch = tecton.BatchSource(
    name='transactions_batch',
    
    batch_config=tecton.SnowflakeConfig(
        database="TECTON_DEMO_DATA",
        schema="FRAUD_DEMO",
        table="TRANSACTIONS",
        timestamp_field="TIMESTAMP"
    )
)

### Inspect data source

In [9]:
transactions_batch.get_columns()

BatchSource 'transactions_batch': Deriving schema.
BatchSource 'transactions_batch': Successfully validated.


['USER_ID',
 'TRANSACTION_ID',
 'CATEGORY',
 'AMT',
 'IS_FRAUD',
 'MERCHANT',
 'MERCH_LAT',
 'MERCH_LONG',
 'TIMESTAMP']

### Create the entity and feature view logic

Now that we have our data source created, we can now create our feature view, which contains both the transformation logic we want to run on our data source as well as orchestration parameters.

An **entity** just tells Tecton what the join key is, in this case the `USER_ID` column. Tecton will check to make sure this column(s) exist after running our transformation logic, so we need to make sure we return a `USER_ID` column from our feature view

In [10]:
user = tecton.Entity(
    name='fraud_user',
    join_keys=['USER_ID']
)

A **transformation** is a piece of Snowflake logic that takes as input some data sources and outputs a table. We can also string multiple (reusable) transformation blocks together.

In [11]:
@tecton.transformation(mode="snowflake_sql")
def transactions_transformation(transactions_batch, amt_to_add):
    return f'''
        SELECT
            TIMESTAMP,
            USER_ID,
            AMT,
            AMT + {amt_to_add} as AMT_PLUS_X
        FROM
            {transactions_batch}
        '''

A **batch_feature_view** ties it all together: we specify the data source, entity, and call the transformation we defined above, plus we add in what storage layer Tecton should write this data to and how frequently Tecton should run this logic against new data.

In [12]:
@tecton.batch_feature_view(
    sources=[tecton.FilteredSource(transactions_batch)],
    entities=[user],
    mode='snowflake_sql',
    online=False,
    offline=False,
    feature_start_time=datetime.datetime(2022, 4, 1),
    batch_schedule=datetime.timedelta(days=1),
    ttl=datetime.timedelta(days=30),
    description='Last user transaction amount (batch calculated)',
)
def last_transaction_amount_batch(transactions_batch):
    return f'''
        SELECT
            TIMESTAMP,
            USER_ID,
            AMT,
            AMT + 2 as AMT_PLUS_X
        FROM
            {transactions_batch}
        '''

## Test our feature view

Now let's see what kind of data we get back from our feature view

In [13]:
tdf = last_transaction_amount_batch.get_historical_features()

BatchFeatureView 'last_transaction_amount_batch': Validating 2 of 3 dependencies. (1 already validated)
    Entity 'fraud_user': Successfully validated.
    Transformation 'last_transaction_amount_batch': Successfully validated.
BatchFeatureView 'last_transaction_amount_batch': Successfully validated.


INFO - 02/28/2023 02:51:32 PM - FeatureView - 1 Feature View is being computed directly from raw data sources.


Our output is a Tecton dataframe and we're going to save this as a Snowpark dataframe. [Snowpark](https://www.snowflake.com/en/data-cloud/snowpark/) is like Spark on Snowflake (get it?) and the Spark dataframe methods you might be familiar with you can use here. Working in Snowpark is a good idea if you're going to be examining a Tecton query that could be bringing back a large set of results.

In [16]:
sdf = tdf.to_snowpark()

In [17]:
type(sdf)

snowflake.snowpark.dataframe.DataFrame

In [18]:
sdf.limit(10).show()

--------------------------------------------------------------------------------
|"TIMESTAMP"                 |"USER_ID"          |"AMT"   |"AMT_PLUS_X"        |
--------------------------------------------------------------------------------
|2022-04-08 06:32:38.392967  |user_212730160038  |64.48   |66.48               |
|2022-04-08 06:32:40.793482  |user_722584453020  |4.16    |6.16                |
|2022-04-08 06:32:43.953836  |user_939970169861  |66.4    |68.4                |
|2022-04-08 06:32:46.167030  |user_699668125818  |1.23    |3.23                |
|2022-04-08 06:32:47.728294  |user_939970169861  |1.28    |3.2800000000000002  |
|2022-04-08 06:32:50.484319  |user_461615966685  |89.61   |91.61               |
|2022-04-08 06:32:52.296845  |user_574612776685  |157.91  |159.91              |
|2022-04-08 06:32:54.209728  |user_222506789984  |45.76   |47.76               |
|2022-04-08 06:32:56.107644  |user_337750317412  |2.3     |4.3                 |
|2022-04-08 06:32:58.344456 

You can always convert a Snowpark dataframe into a Pandas dataframe and continue. Or, you can convert a Tecton dataframe directly to a pandas dataframe.. just make sure you're not unloading an entire table!

In [19]:
pdf = sdf.limit(10).to_pandas()

In [20]:
type(pdf)

pandas.core.frame.DataFrame

In [21]:
pdf

,TIMESTAMP,USER_ID,AMT,AMT_PLUS_X
0,2022-04-08 06:32:38.392967,user_212730160038,64.48,66.48
1,2022-04-08 06:32:40.793482,user_722584453020,4.16,6.16
2,2022-04-08 06:32:43.953836,user_939970169861,66.40,68.40
3,2022-04-08 06:32:46.167030,user_699668125818,1.23,3.23
4,2022-04-08 06:32:47.728294,user_939970169861,1.28,3.28
5,2022-04-08 06:32:50.484319,user_461615966685,89.61,91.61
6,2022-04-08 06:32:52.296845,user_574612776685,157.91,159.91
7,2022-04-08 06:32:54.209728,user_222506789984,45.76,47.76
8,2022-04-08 06:32:56.107644,user_337750317412,2.30,4.30
9,2022-04-08 06:32:58.344456,user_934384811883,115.32,117.32


# Append our feature view to a remote feature service

## Get v1 of our feature service remotely from Tecton

In [22]:
ws = tecton.get_workspace('workshop-jack')

In [23]:
ws.list_feature_services()

['fraud_detection_feature_service']

In [24]:
fs = ws.get_feature_service('fraud_detection_feature_service')

In [25]:
fs.summary()

Name,fraud_detection_feature_service
Owner,
Last Updated By,jack@tecton.ai
Description,
Entities,"['fraud_user', 'category']"
Online Serving,Enabled
Logging,Disabled
Online Join Keys,"USER_ID, CATEGORY"
Offline Join Keys,"USER_ID, CATEGORY"
Request Context Keys,
Defined In,feature_services/fraud_detection.py


## Modify our new feature view to use the existing data source

We only created the data source above from scratch because it's worth going seeing what the whole flow looks like. But doing this for real, let's use what we've already got!

In [26]:
ws.list_data_sources()

['transactions', 'users']

In [27]:
ws.list_entities()

['category', 'fraud_user', 'merchant']

In [28]:
ws.get_entity('fraud_user').summary()

Name,fraud_user
Workspace,workshop-jack
Description,A user of the platform
Created At,2023-02-28 00:58:05 UTC
Owner,matt@tecton.ai
Last Modified By,jack@tecton.ai
Source Filename,entities.py
Tags,release production
Join Keys,USER_ID


In [29]:
@tecton.batch_feature_view(
    sources=[tecton.FilteredSource(ws.get_data_source('transactions'))],
    entities=[ws.get_entity('fraud_user')],
    mode='snowflake_sql',
    online=False,
    offline=False,
    feature_start_time=datetime.datetime(2022, 4, 1),
    batch_schedule=datetime.timedelta(days=1),
    ttl=datetime.timedelta(days=30),
    description='Last user transaction amount (batch calculated)'
)
def last_transaction_amount_batch_remote(transactions_batch):
    return f'''
        SELECT
            TIMESTAMP,
            USER_ID,
            AMT,
            AMT + 2 as AMT_PLUS_X
        FROM
            {transactions_batch}
        '''

## Create v2 of the feature service by appending our new (remote) feature view

In [48]:
fs_v2 = tecton.FeatureService(
    name='fraud_detection_feature_service_v2',
    features=fs.features+[last_transaction_amount_batch_remote]
)

In [50]:
fs_v2

FeatureService(info=TectonObjectInfo(id='42f8cbed0b72305969e053f3ff99550f', name='fraud_detection_feature_service_v2', description=None, tags={}, owner=None, workspace=None), _feature_references=[FeatureReference(feature_definition=BatchFeatureView('user_transaction_metrics'), namespace='user_transaction_metrics', features=('TRANSACTION_SUM_1D_1D', 'TRANSACTION_SUM_3D_1D', 'TRANSACTION_SUM_7D_1D', 'TRANSACTION_SUM_40D_1D', 'AMT_MEAN_1D_1D', 'AMT_MEAN_3D_1D', 'AMT_MEAN_7D_1D', 'AMT_MEAN_40D_1D'), override_join_keys={'USER_ID': 'USER_ID'}), FeatureReference(feature_definition=BatchFeatureView('user_category_count'), namespace='user_category_count', features=('TRANSACTION_SUM_1D_1D', 'TRANSACTION_SUM_3D_1D', 'TRANSACTION_SUM_7D_1D', 'TRANSACTION_SUM_40D_1D'), override_join_keys={'USER_ID': 'USER_ID', 'CATEGORY': 'CATEGORY'}), FeatureReference(feature_definition=BatchFeatureView('last_transaction_amount_batch_remote'), namespace='last_transaction_amount_batch_remote', features=None, overri

## Build a spine dataframe

In [31]:
spine = pd.DataFrame([
    ["user_469998441571", 'shopping_net', pd.Timestamp('2022-04-10 00:00Z'), True],
    ["user_469998441571", 'shopping_net', pd.Timestamp('2022-04-11 00:00Z'), False],
    ["user_469998441571", 'shopping_net', pd.Timestamp('2022-04-11 01:00Z'), False],
    ["user_445755474326", 'misc_pos', pd.Timestamp('2022-04-10 00:00Z'), False],
    ["user_445755474326", 'misc_pos', pd.Timestamp('2022-04-11 00:00Z'), False],
    ["user_445755474326", 'misc_pos', pd.Timestamp('2022-04-11 01:00Z'), False]
], columns=['USER_ID', 'CATEGORY', 'TIMESTAMP', 'is_fraud'])

spine

,USER_ID,CATEGORY,TIMESTAMP,is_fraud
0,user_469998441571,shopping_net,2022-04-10 00:00:00+00:00,True
1,user_469998441571,shopping_net,2022-04-11 00:00:00+00:00,False
2,user_469998441571,shopping_net,2022-04-11 01:00:00+00:00,False
3,user_445755474326,misc_pos,2022-04-10 00:00:00+00:00,False
4,user_445755474326,misc_pos,2022-04-11 00:00:00+00:00,False
5,user_445755474326,misc_pos,2022-04-11 01:00:00+00:00,False


## `get_historical_features` (v1)

In [32]:
tdf = fs.get_historical_features(spine)

INFO - 02/28/2023 02:55:18 PM - FeatureService - Computing historical feature values for 2 Feature Views.
INFO - 02/28/2023 02:55:18 PM - FeatureView - 2 Feature Views are being computed directly from raw data sources.
WARNING - 02/28/2023 02:55:22 PM - snowflake.snowpark - create_temp_table is deprecated. We still respect this parameter when it is True but please consider using `table_type="temporary"` instead.


In [34]:
tdf.to_snowpark().show()

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"USER_ID"          |"CATEGORY"    |"TIMESTAMP"          |"is_fraud"  |"USER_TRANSACTION_METRICS__TRANSACTION_SUM_1D_1D"  |"USER_TRANSACTION_METRICS__TRANSACTION_SUM_3D_1D"  |"USER_TRANSACTION_METRICS__TRANSACTION_SUM_7D_1D"  |"USER_TRANSACTION_METRICS__TRANSACTION_SUM_40D_1D"  |"USER_TRANSACTION_METRICS__AMT_MEAN_1D_1D"  |"USER_TRANSACTION_METRIC

In [35]:
tdf.to_pandas()

,USER_ID,CATEGORY,TIMESTAMP,is_fraud,USER_TRANSACTION_METRICS__TRANSACTION_SUM_1D_1D,USER_TRANSACTION_METRICS__TRANSACTION_SUM_3D_1D,USER_TRANSACTION_METRICS__TRANSACTION_SUM_7D_1D,USER_TRANSACTION_METRICS__TRANSACTION_SUM_40D_1D,USER_TRANSACTION_METRICS__AMT_MEAN_1D_1D,USER_TRANSACTION_METRICS__AMT_MEAN_3D_1D,USER_TRANSACTION_METRICS__AMT_MEAN_7D_1D,USER_TRANSACTION_METRICS__AMT_MEAN_40D_1D,USER_CATEGORY_COUNT__TRANSACTION_SUM_1D_1D,USER_CATEGORY_COUNT__TRANSACTION_SUM_3D_1D,USER_CATEGORY_COUNT__TRANSACTION_SUM_7D_1D,USER_CATEGORY_COUNT__TRANSACTION_SUM_40D_1D
0,user_469998441571,shopping_net,2022-04-10 00:00:00,True,767,1508,1508,1508,60.329335,61.626545,61.626545,61.626545,65,116,116,116
1,user_469998441571,shopping_net,2022-04-11 00:00:00,False,683,2160,2191,2191,57.327950,60.497157,60.286545,60.286545,51,165,167,167
2,user_469998441571,shopping_net,2022-04-11 01:00:00,False,683,2160,2191,2191,57.327950,60.497157,60.286545,60.286545,51,165,167,167
3,user_445755474326,misc_pos,2022-04-10 00:00:00,False,482,1063,1063,1063,60.277863,59.972117,59.972117,59.972117,44,83,83,83
4,user_445755474326,misc_pos,2022-04-11 00:00:00,False,530,1562,1593,1593,61.344057,60.582881,60.428569,60.428569,37,120,120,120
5,user_445755474326,misc_pos,2022-04-11 01:00:00,False,530,1562,1593,1593,61.344057,60.582881,60.428569,60.428569,37,120,120,120


## `get_historical_features` v2

In [55]:
tdf = fs_v2.get_historical_features(spine)

FeatureService 'fraud_detection_feature_service_v2': Validating 1 of 3 dependencies. (2 already validated)
    BatchFeatureView 'last_transaction_amount_batch_remote': Validating 1 of 3 dependencies. (2 already validated)
        Transformation 'last_transaction_amount_batch_remote': Successfully validated.
    BatchFeatureView 'last_transaction_amount_batch_remote': Successfully validated.
FeatureService 'fraud_detection_feature_service_v2': Successfully validated.


INFO - 02/28/2023 03:03:46 PM - FeatureService - Computing historical feature values for 3 Feature Views.
INFO - 02/28/2023 03:03:46 PM - FeatureView - 3 Feature Views are being computed directly from raw data sources.


In [56]:
pdf = tdf.to_pandas()

In [57]:
pdf.columns

Index(['USER_ID', 'CATEGORY', 'TIMESTAMP', 'is_fraud',
       'USER_TRANSACTION_METRICS__TRANSACTION_SUM_1D_1D',
       'USER_TRANSACTION_METRICS__TRANSACTION_SUM_3D_1D',
       'USER_TRANSACTION_METRICS__TRANSACTION_SUM_7D_1D',
       'USER_TRANSACTION_METRICS__TRANSACTION_SUM_40D_1D',
       'USER_TRANSACTION_METRICS__AMT_MEAN_1D_1D',
       'USER_TRANSACTION_METRICS__AMT_MEAN_3D_1D',
       'USER_TRANSACTION_METRICS__AMT_MEAN_7D_1D',
       'USER_TRANSACTION_METRICS__AMT_MEAN_40D_1D',
       'USER_CATEGORY_COUNT__TRANSACTION_SUM_1D_1D',
       'USER_CATEGORY_COUNT__TRANSACTION_SUM_3D_1D',
       'USER_CATEGORY_COUNT__TRANSACTION_SUM_7D_1D',
       'USER_CATEGORY_COUNT__TRANSACTION_SUM_40D_1D',
       'LAST_TRANSACTION_AMOUNT_BATCH_REMOTE__AMT',
       'LAST_TRANSACTION_AMOUNT_BATCH_REMOTE__AMT_PLUS_X'],
      dtype='object')

In [60]:
pdf[['USER_ID', 'CATEGORY', 
     'LAST_TRANSACTION_AMOUNT_BATCH_REMOTE__AMT', 'LAST_TRANSACTION_AMOUNT_BATCH_REMOTE__AMT_PLUS_X', 
     'TIMESTAMP']]

,USER_ID,CATEGORY,LAST_TRANSACTION_AMOUNT_BATCH_REMOTE__AMT,LAST_TRANSACTION_AMOUNT_BATCH_REMOTE__AMT_PLUS_X,TIMESTAMP
0,user_469998441571,shopping_net,88.79,90.79,2022-04-10 00:00:00
1,user_469998441571,shopping_net,38.96,40.96,2022-04-11 00:00:00
2,user_469998441571,shopping_net,39.65,41.65,2022-04-11 01:00:00
3,user_445755474326,misc_pos,1.56,3.56,2022-04-10 00:00:00
4,user_445755474326,misc_pos,107.95,109.95,2022-04-11 00:00:00
5,user_445755474326,misc_pos,29.26,31.26,2022-04-11 01:00:00


# Apply our new feature view + feature service into our repository!

So far, all of our development has been done locally in our Jupyter instance. We're happy with our work so far and we want to push it to a workspace in Tecton so we can do additional integration testing and/or push to production:

The steps are:
* Copy and paste the feature view and feature service definitions into the repository's `.py` files
* Log into your cluster (`tecton login https://...`) and select or create a workspace (`tecton workspace create ...` or `tecton workspace select ...`)
* Run a `tecton plan` to inspect your features
* Once satisfied, run `tecton apply` to push your features to the workspace

Now let's test our feature service!

In [ ]:
fs_v2_remote = ws.get_feature_service('fraud_detection_feature_service_v2')

In [ ]:
# We'll use the spine dataframe we defined above
tdf = fs_v2_remote.get_historical_features(spine)

In [ ]:
tdf.to_snowpark().show()

# Get real-time features from your REST API

Once you've tested your features to make sure they work (and check out our Unit Testing capabilities!), if you need real-time serving, there are just a couple more steps:

* Create or select a **live** workspace. This is a workspace with materialization switched on.
* Wait for your features to materialize. You can check the Web UI for more details here.
* Create or use a service account and grant it at least `Consumer` level privileges
* Use the API key associated with the service account to retrieve features.

You can use any language to retrieve features, since it's just a REST API call. We suggest using native http requests libraries in your language (such as `requests` perhaps with `asyncio` with Python), or if you're on Java, check out our high-performance REST API client [here](https://github.com/tecton-ai/tecton-http-client-java) (also available on maven).

In [61]:
import requests
import json

Let's formulate the request data packet. You can use our [API reference documentation](https://docs.tecton.ai/http-api) and [this docs page](https://docs.tecton.ai/docs/reading-feature-data/reading-feature-data-for-inference/reading-online-features-for-inference-using-the-http-api/#metadata-options-for-the-http-api) for information on the metadata you can retrieve on your features.

In [62]:
request_data = json.dumps(
    {
        'params': {
            'feature_service_name': 'fraud_detection_feature_service',
            'join_key_map': {
                'USER_ID': 'user_469998441571',
                'CATEGORY': 'fraud'
            },
            'workspace_name': 'tecton-training-102-live',
            'metadata_options': {
                "include_names": True,
                "include_effective_times": True,
                "include_data_types": True,
                "include_slo_info": True,
                'include_serving_status': True
            } 
        }
    }

)

Let's make the post request (we're reading the `REST_API_KEY` variable from the .env file and/or environment variable)

In [63]:
r = requests.post(
    url='https://dev-snowflake.tecton.ai/api/v1/feature-service/get-features',
    headers={
        'Authorization': f"Tecton-key {os.environ['REST_API_KEY']}"
    },
    data=request_data
)

Now we examine the output

In [64]:
r.status_code

200

In [65]:
res = json.loads(r.text)

In [66]:
res['result']

{'features': [None,
  None,
  None,
  None,
  54.569518248175186,
  57.43892337536372,
  60.66241346470824,
  60.405883859948766,
  '685',
  '2062',
  '27301',
  '4684']}

This next line is the full result that includes a lot of metadata. In a production environment, you'll probably want to omit the metadata for every call.

In [67]:
res

{'result': {'features': [None,
   None,
   None,
   None,
   54.569518248175186,
   57.43892337536372,
   60.66241346470824,
   60.405883859948766,
   '685',
   '2062',
   '27301',
   '4684']},
 'metadata': {'features': [{'name': 'user_category_count.TRANSACTION_SUM_1D_1D',
    'effectiveTime': '2023-02-27T00:00:00Z',
    'dataType': {'type': 'int64'},
    'status': 'MISSING_DATA'},
   {'name': 'user_category_count.TRANSACTION_SUM_3D_1D',
    'effectiveTime': '2023-02-27T00:00:00Z',
    'dataType': {'type': 'int64'},
    'status': 'MISSING_DATA'},
   {'name': 'user_category_count.TRANSACTION_SUM_40D_1D',
    'effectiveTime': '2023-02-27T00:00:00Z',
    'dataType': {'type': 'int64'},
    'status': 'MISSING_DATA'},
   {'name': 'user_category_count.TRANSACTION_SUM_7D_1D',
    'effectiveTime': '2023-02-27T00:00:00Z',
    'dataType': {'type': 'int64'},
    'status': 'MISSING_DATA'},
   {'name': 'user_transaction_metrics.AMT_MEAN_1D_1D',
    'effectiveTime': '2023-02-27T00:00:00Z',
    'data